# Download Vessel Counts by Day


In [4]:
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import colors,colorbar
%matplotlib inline
import csv 
import math
from math import radians, cos, sin, asin, sqrt
from scipy import stats
import math
import cPickle 

In [5]:
import bq  
client = bq.Client.Get()

In [6]:
def Query(q):
    t0 = time.time()
    answer = client.ReadTableRows(client.Query(q)['configuration']['query']['destinationTable'])
    print 'Query time: ' + str(time.time() - t0) + ' seconds.'
    return answer

In [12]:
def query_date(thedate):
    q = '''
    SELECT
      integer(FLOOR(first_lat*10)) lat_bin,
      integer(FLOOR(first_lon*10)) lon_bin,
      integer(FLOOR(avg_lat*10)) lat_bin_avg,
      integer(FLOOR(avg_lon*10)) lon_bin_avg,
      satellite_positions sat_positions,
      terrestrial_positions terrestrial_positions,
      positions_weighted,
      avg_speed,
      slow_pings,
      mmsi
    FROM
    (SELECT
      mmsi,
      SUM( CASE WHEN speed = 0 OR (speed<=2 AND type IN (18, 19)) THEN 180 
         WHEN (speed > 0 AND speed <14 AND type IN (1,2,3)AND turn = 0 )
             OR (speed>2 AND type IN (18,19)) THEN 10 
          when speed>0 and speed<14 and type in (1,2,3) and turn !=0 then 3.3
          when speed>=14 and speed<23 and type in (1,2,3) and turn = 0 then 6
          when type in (1,2,3) and (speed>=23 or (speed>=14 and turn !=0)) then 2
          END) positions_weighted,
       first(lat) first_lat,
       first(lon) first_lon,
       avg(lat) avg_lat,
       avg(lon) avg_lon,
       max(lat) max_lat,
       min(lat) min_lat,
       max(lon) max_lon,
       min(lon) min_lon,
       avg(speed) avg_speed,
       sum(if( (speed=0 and type in (1,2,3)) or (speed<2 and type in (18,19)),1,0 )) slow_pings,
       sum( if(REGEXP_REPLACE(tagblock_station, 'u', '') IN ('rORBCOMM000',
            'rORBCOMM01',
            'rORBCOMM008',
            'rORBCOMM009',
            'rORBCOMM010'),1,0)) terrestrial_positions,
       sum( if(REGEXP_REPLACE(tagblock_station, 'u', '') not IN ('rORBCOMM000',
            'rORBCOMM01',
            'rORBCOMM008',
            'rORBCOMM009',
            'rORBCOMM010'),1,0)) satellite_positions,       
    FROM
      [pipeline_normalize.'''+thedate+''']
    WHERE
      type IN (1,2,3,18,19) and lat is not null and lon is not null and speed is not null and turn is not null
    group by mmsi
    )
      where
      max_lat - min_lat <5
      AND (max_lon - min_lon < 10
        OR first_lon > 170
        OR first_lon < -170)
      AND mmsi IN (select mmsi from
    [scratch_david_gapanalysis.good_mmsi_2015_1000pings])
    '''

    positions = Query(q)
    cPickle.dump(positions, open('../../data/density/'+thedate+'_v2_vessels.p', 'wb')) 

In [15]:
for i in range(1,32):
    thedate = "201503"
    if i<10:
        thedate += "0"+str(i)
    else:
        thedate += str(i)
    print thedate
    query_date(thedate)
    

20150301
Waiting on bqjob_rc5d1d36a0673555_00000153a58cea75_85 ... (14s) Current status: DONE   
Query time: 34.4649701118 seconds.
20150302
Waiting on bqjob_r129120c3ab62cee1_00000153a58d78fb_86 ... (15s) Current status: DONE   
Query time: 52.4378290176 seconds.
20150303
Waiting on bqjob_r1fd315b1681a4fad_00000153a58e54d4_87 ... (15s) Current status: DONE   
Query time: 46.6302821636 seconds.
20150304
Waiting on bqjob_r50de679d8e60065a_00000153a58f19ae_88 ... (23s) Current status: DONE   
Query time: 59.0982401371 seconds.
20150305
Waiting on bqjob_r4240430d55b85765_00000153a5900f9c_89 ... (15s) Current status: DONE   
Query time: 47.1999361515 seconds.
20150306
Waiting on bqjob_r2e32496f8d58ab02_00000153a590d7dc_90 ... (14s) Current status: DONE   
Query time: 47.9087018967 seconds.
20150307
Waiting on bqjob_r4b5766a43c5a75f1_00000153a591a1c4_91 ... (14s) Current status: DONE   
Query time: 45.9500920773 seconds.
20150308
Waiting on bqjob_recf75325fc14252_00000153a5926412_92 ... (9s

In [16]:
import csv
for i in range(1,7):
    thedate = "201501"
    if i<10:
        thedate += "0"+str(i)
    else:
        thedate += str(i)
    print thedate   
    positions = cPickle.load(open('../../data/density/'+thedate+'_v2_vessels.p', 'rb'))
    with open('../../data/density/'+thedate+'_v2_vessels.csv', 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(["lat_bin","lon_bin","lat_bin_avg","lon_bin_avg","sat_positions",
                         "terrestrial_positions","positions_weighted",
                        "avg_speed","slow_pings","mmsi"])
        writer.writerows(positions)

20150101
20150102
20150103
20150104
20150105
20150106
